# 04.1c: Interactive Sky Map

**Goal:** Interactive Plotly sky map with token ID on hover.

Load pre-computed spherical coordinates and create an interactive visualization where:
- Mouse over any token to see its token ID
- Pan, zoom, and explore the sky
- Note interesting token IDs for deeper inspection in 05.2a

This is an **analyzer** notebook - run after 04.1b has generated spherical coords.

## Parameters

In [16]:
TENSOR_DIR = "../data/tensors"

# Which spherical coordinate tensor to load
SPHERICAL_COORDS_FILE = "spherical_coords_pc1_pc2_pc3.safetensors"

# Visualization parameters
PROJECTION = 'rectangular'  # 'rectangular' or 'mollweide'
HEADING = 0                  # Rotation offset (degrees)
POINT_SIZE = 2               # Size of scatter points
POINT_ALPHA = 0.5           # Opacity (0-1)

## Imports

In [17]:
import torch
import numpy as np
import plotly.graph_objects as go
from safetensors.torch import load_file
from pathlib import Path

print("Imports loaded successfully.")

Imports loaded successfully.


## Step 1: Load Spherical Coordinates

In [18]:
coords_path = Path(TENSOR_DIR) / SPHERICAL_COORDS_FILE
coords = load_file(coords_path)

r = coords['r']
phi_deg = coords['phi_deg']
theta_deg = coords['theta_deg']
theta_flat = coords['theta_flat']

N = len(r)

print(f"Loaded spherical coordinates from: {SPHERICAL_COORDS_FILE}")
print(f"  Tokens: {N:,}")
print(f"  r range: [{r.min().item():.6f}, {r.max().item():.6f}] gamma units")
print(f"  phi range: [{phi_deg.min().item():.2f}°, {phi_deg.max().item():.2f}°]")
print(f"  theta range: [{theta_deg.min().item():.2f}°, {theta_deg.max().item():.2f}°]")
print(f"  theta_flat range: [{theta_flat.min().item():.2f}°, {theta_flat.max().item():.2f}°]")

Loaded spherical coordinates from: spherical_coords_pc1_pc2_pc3.safetensors
  Tokens: 151,936
  r range: [0.002472, 1.095257] gamma units
  phi range: [-179.99°, 180.00°]
  theta range: [-89.92°, 89.68°]
  theta_flat range: [-90.00°, 90.00°]


## Step 2: Apply Heading Rotation

In [19]:
# Rotate longitude by heading
phi_rotated = phi_deg - HEADING

# Wrap to [-180, 180]
phi_rotated = torch.where(phi_rotated > 180, phi_rotated - 360, phi_rotated)
phi_rotated = torch.where(phi_rotated < -180, phi_rotated + 360, phi_rotated)

print(f"\nApplied heading rotation: {HEADING}°")
print(f"  Rotated phi range: [{phi_rotated.min().item():.2f}°, {phi_rotated.max().item():.2f}°]")


Applied heading rotation: 0°
  Rotated phi range: [-179.99°, 180.00°]


## Step 3: Create Interactive Sky Map

In [20]:
print(f"\nCreating interactive sky map...")
print(f"  Projection: {PROJECTION}")
print(f"  Point size: {POINT_SIZE}")
print(f"  Opacity: {POINT_ALPHA}")

# Convert to numpy for Plotly
phi_np = phi_rotated.cpu().numpy()
theta_flat_np = theta_flat.cpu().numpy()
token_ids = np.arange(N)

# Create scatter plot
fig = go.Figure()

fig.add_trace(go.Scattergl(
    x=phi_np,
    y=theta_flat_np,
    mode='markers',
    marker=dict(
        size=POINT_SIZE,
        color='lightblue',
        opacity=POINT_ALPHA,
        line=dict(width=0)
    ),
    text=token_ids,
    hovertemplate='Token ID: %{text}<extra></extra>',
    showlegend=False
))

# Layout
title = f"Sky Map (Heading={HEADING}°, {PROJECTION.title()})"
if HEADING != 0:
    title += f" - Lon shifted by {-HEADING}°"

fig.update_layout(
    title=title,
    xaxis_title="Longitude (degrees)",
    yaxis_title="Latitude (CDF-flattened, degrees)",
    plot_bgcolor='black',
    paper_bgcolor='white',
    xaxis=dict(
        range=[-180, 180],
        dtick=30,
        gridcolor='rgba(255,255,255,0.2)',
        zeroline=False
    ),
    yaxis=dict(
        range=[-90, 90],
        dtick=30,
        gridcolor='rgba(255,255,255,0.2)',
        zeroline=False
    ),
    width=1400,
    height=700,
    hovermode='closest'
)

fig.show()

print("\nInteractive sky map created!")
print("Hover over tokens to see their IDs.")


Creating interactive sky map...
  Projection: rectangular
  Point size: 2
  Opacity: 0.5



Interactive sky map created!
Hover over tokens to see their IDs.


## Summary

Interactive sky map for exploring token space.

**Workflow:**
1. Hover over interesting features to note token IDs
2. Use 05.2a to inspect individual tokens by ID
3. Discover structures and patterns

**Tips:**
- Adjust `HEADING` to center different longitudes
- Adjust `POINT_SIZE` and `POINT_ALPHA` for dense regions
- Use Plotly zoom/pan tools to explore details